# **[MC906] Projeto Final**: Detecção de Desastres

O objetivo desse projeto é construir e avaliar modelos de aprendizado de máquina que classifiquem quais Tweets são sobre desastres reais e quais não são.

## **Acessar Diretório do Projeto**

Esse Notebook assume que você está executando o código dentro da pasta `Projeto Final/Código`, que contém todos os códigos fontes relativos a esse trabalho. Para acessar o diretório no Colab é preciso criar um atalho em seu Drive (right click no diretório -> "Adicionar atalho ao Google Drive") e executar as células abaixo:

In [ ]:
# Conectar ao Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
# Diretório do Projeto (/content/drive/My Drive/{path até Projeto Final/Código}), 
# dependendo da onde se localiza o atalho no seu Drive
% cd '/content/drive/My Drive/[MC906] Introdução à Inteligência Artificial/Projeto Final/Código'
! ls

/content/drive/.shortcut-targets-by-id/1HmHC25ZqX3hUlCsRT-S0qiSsjwf10jLn/[MC906] Introdução à Inteligência Artificial/Projeto Final
'Attention CNN'			'Cópia de best_model_BERT_back.h5'
'BERT Backup'			 Dataset
'BERT Inicial.ipynb'		'Dense Neural Networks'
'BERT Melhor'			 Glove
 best_model_attention.h5	'Logistic Regression and SVM'
 best_model_BERT_back.h5	'Melhor Pré-Processamento'
 best_model_BERT.h5		 __pycache__
 best_model_CNN.h5		'Quasi-Recurrent Networks'
 best_model_DNN.h5		'Recurrent Neural Networks'
 best_model_RNN_glove.h5	 Relatório
 best_model_RNN.h5		 tokenization.py
 best_model_RNN_pool.h5		 utils.py
'Convolutional Neural Network'


## **Dependências:**


In [ ]:
# Imports de pacotes instalados
import pandas as pd
from os.path import join, exists
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Activation, Dense, Input, Embedding
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

# Instalações
!pip install sentencepiece # Usado em tokenization.py 

# Imports locais
from utils import *
import tokenization

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 1.1MB 2.7MB/s 


## **Dataset:**

Utilizamos um *dataset* disponível no site [Kaggle](https://www.kaggle.com/c/nlp-getting-started/data) (em inglês). Cada tweet apresenta três atributos: seu conteúdo (`text`), uma palavra-chave (`keyword`, opcional) e a localização da onde foi enviado (`location`, opcional). Como só usaremos o texto, removemos os dois últimos.

In [ ]:
# Ler e limpar dados (removendo colunas do id, keyword e location)
train = pd.read_csv("../Dataset/train.csv", dtype={'id': np.int16, 'target': np.int8})
train = train.drop(['id','keyword','location'], axis=1)

# Imprimir alguns dados
print(train.head())
vals = train.groupby('target').count()
print("\nSome General insights:")
print(f"Figure of Speech: {vals.iloc[0]['text']*100/len(train):.2f}%")
print(f"Actual Accidents: {vals.iloc[1]['text']*100/len(train):.2f}%")

                                                text  target
0  Our Deeds are the Reason of this #earthquake M...       1
1             Forest fire near La Ronge Sask. Canada       1
2  All residents asked to 'shelter in place' are ...       1
3  13,000 people receive #wildfires evacuation or...       1
4  Just got sent this photo from Ruby #Alaska as ...       1

Some General insights:
Figure of Speech: 57.03%
Actual Accidents: 42.97%


## **Pré-Processamento:**

Inicialmente apenas removemos as pontuações e caracteres especiais de cada texto. Depois, com o próprio tokenizer do BERT (modelo que usamos nesse Notebook), normalizamos o texto e o dividimos em tokens, adequando o conjunto de dados para treinamento. 

In [ ]:
# Limpar texto
print(f"Raw Tweet:\n\t",train.text[1])
train.text = train.text.apply(clean_up)
print("\nRemoved Punctuation and Special Chars:\n\t", train.text[1])

Raw Tweet:
	 Forest fire near La Ronge Sask. Canada

Removed Punctuation and Special Chars:
	 Forest fire near La Ronge Sask Canada


In [ ]:
%%time
# Carregar camada 
bert_layer = hub.KerasLayer(
    'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2', 
    trainable=True
)

CPU times: user 8.68 s, sys: 2.03 s, total: 10.7 s
Wall time: 15.3 s


In [ ]:
# Salvar tokenizer
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def encode(texts, tokenizer, max_len=200):
  ''' Função que aplica a tokenização WordPiece no conjunto `texts`, truncando
  os textos em `max_len` caracteres e adicionando os tokens [CLS] e [SEP] em
  seus extremos. '''

  all_tokens = []
  all_masks = []
  all_segments = []
  
  for text in texts:
      text = tokenizer.tokenize(text)
          
      text = text[:max_len-2]
      input_sequence = ["[CLS]"] + text + ["[SEP]"]
      pad_len = max_len - len(input_sequence)
      
      tokens = tokenizer.convert_tokens_to_ids(input_sequence)
      tokens += [0] * pad_len
      pad_masks = [1] * len(input_sequence) + [0] * pad_len
      segment_ids = [0] * max_len
      
      all_tokens.append(tokens)
      all_masks.append(pad_masks)
      all_segments.append(segment_ids)
  
  return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
# Preparar textos para treinamento
train_input = encode(train.text.values, tokenizer)
train_labels = train.target.values

## **Modelo**: BERT

BERT é um modelo não-supervisionado e bidirecional pré-treinado em uma coleção de escritos enorme (Wikipedia + BookCorpus) que pode ser afinado (*fine-tuned*) para tarefas específicas. Por representar o estado da arte em NLP, resolvemos testar nesse projeto.

In [ ]:
def NN(bert_layer, max_len=200):
  ''' Função que constrói o modelo BERT. '''

  input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
  input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
  segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
  _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
  clf_output = sequence_output[:, 0, :]
  out = Dense(1, activation='sigmoid')(clf_output)
  model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
  return model

In [ ]:
# Construir e compilar modelo
neural_network = NN(bert_layer)
neural_network.summary()
neural_network.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 200)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 200)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [ ]:
# Treinar no dataset pré-processado
callbacks = [ModelCheckpoint(monitor='val_loss', filepath='./Modelos/best_model_BERT.h5', save_best_only=True)]
history = neural_network.fit(train_input, train_labels, batch_size=32, epochs=15, validation_split=0.1, callbacks=callbacks)

Epoch 1/15
215/215 [==============================] - 15297s 71s/step - loss: 0.4800 - accuracy: 0.7813 - val_loss: 0.5658 - val_accuracy: 0.7612
Epoch 2/15
215/215 [==============================] - 15341s 71s/step - loss: 0.3989 - accuracy: 0.8304 - val_loss: 0.3724 - val_accuracy: 0.8412
Epoch 3/15
177/215 [=======================>......] - ETA: 43:55 - loss: 0.3614 - accuracy: 0.8517

## **Avaliação**

Utilizamos o conjunto de teste mergeado com as respostas vazadas para avaliar se o modelo generaliza bem pro problema.

In [ ]:
# Carregar modelo treinado
if exists('./Modelos/best_model_BERT.h5'):
    neural_network = load_model('./Modelos/best_model_BERT.h5', custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
# Salvar conjunto de testes com as respostas
test = pd.read_csv("../Dataset/test_with_targets.csv", dtype={'id': np.int16, 'target': np.int8})
test = test.drop(['id','keyword','location'], axis=1)

# Aplicar pré-processamento
test.text = test.text.apply(clean_up)
test_input = encode(test.text.values, tokenizer)
test_labels = test.target.values

In [ ]:
# Avaliar modelo
neural_network.evaluate(x=test_input, y=test_labels)

102/102 [==============================] - 2197s 22s/step - loss: 0.4003 - accuracy: 0.8302


[0.4002986252307892, 0.8302175998687744]